## Semantic segmentation

This ipython notebook recreates the semantic segmentation figures. First we need to load some packages

In [1]:
import h5py
import tifffile as tiff
from keras.backend.common import _UID_PREFIXES
from skimage.measure import label, regionprops
from cnn_functions import nikon_getfiles, get_image, run_models_on_directory, get_image_sizes, segment_nuclei, segment_cytoplasm, dice_jaccard_indices
from model_zoo import sparse_bn_feature_net_61x61 as cyto_fn
from model_zoo import sparse_bn_feature_net_61x61 as nuclear_fn
from skimage import segmentation
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42


tifffile.py:156: UserWarning: failed to import the optional _tifffile C extension module.
Loading of some compressed images will be slow.
Tifffile.c can be obtained at http://www.lfd.uci.edu/~gohlke/
  "failed to import the optional _tifffile C extension module.\n"
Using Theano backend.


Next, lets process the images with our trained conv-nets

In [ ]:
"""
Load data
"""
direc_name = '/Users/nicolasquach/sherlock_home/DeepCell2/testing_data/MCF10A_3T3/set1'
data_location = os.path.join(direc_name, 'RawImages')
cyto_location = os.path.join(direc_name, 'Cytoplasm')
nuclear_location = os.path.join(direc_name, 'Nuclear')
mask_location = os.path.join(direc_name, 'Masks')

cyto_channel_names = ['Phase', 'DAPI']
nuclear_channel_names = ['DAPI']

trained_network_cyto_directory = "/Users/nicolasquach/sherlock_home/DeepCell2/trained_networks/MCF10A_3T3_semantic"
trained_network_nuclear_directory = "/Users/nicolasquach/sherlock_home/DeepCell2/trained_networks/Nuclear"

cyto_prefix = "2016-07-28_MCF10A_3T3_all_semantic_61x61_bn_feature_net_61x61_semantic_all_"
nuclear_prefix = "2016-07-12_nuclei_all_61x61_bn_feature_net_61x61_"

win_cyto = 30
win_nuclear = 30

image_size_x, image_size_y = get_image_sizes(data_location, nuclear_channel_names)

"""
Define model
"""

list_of_cyto_weights = []
for j in xrange(5):
	cyto_weights = os.path.join(trained_network_cyto_directory,  cyto_prefix + str(j) + ".h5")
	list_of_cyto_weights += [cyto_weights]

list_of_nuclear_weights = []
for j in xrange(5):
	nuclear_weights = os.path.join(trained_network_nuclear_directory,  nuclear_prefix + str(j) + ".h5")
	list_of_nuclear_weights += [nuclear_weights]

"""
Run model on directory
"""

cytoplasm_predictions = run_models_on_directory(data_location, cyto_channel_names, cyto_location, n_features = 4, model_fn = cyto_fn, 
	list_of_weights = list_of_cyto_weights, image_size_x = image_size_x, image_size_y = image_size_y, 
	win_x = win_cyto, win_y = win_cyto, std = False, split = False)

nuclear_predictions = run_models_on_directory(data_location, nuclear_channel_names, nuclear_location, model_fn = nuclear_fn, 
	list_of_weights = list_of_nuclear_weights, image_size_x = image_size_x, image_size_y = image_size_y, 
	win_x = win_nuclear, win_y = win_nuclear, std = False, split = False)

Processing image 1 of 1


Now lets create the masks.

In [3]:
nuclear_masks = segment_nuclei(img = nuclear_predictions, color_image = True, load_from_direc = None, mask_location = mask_location, threshold = 0.75, area_threshold = 100, solidity_threshold = 0.75, eccentricity_threshold = 1)
cyto_combined_predictions = np.zeros(cytoplasm_predictions.shape)
cyto_combined_predictions[:,1,:,:] = cytoplasm_predictions[:,1,:,:] + cytoplasm_predictions[:,2,:,:]
cytoplasm_masks = segment_cytoplasm(img = cyto_combined_predictions, load_from_direc = None, color_image = True, nuclear_masks = nuclear_masks, mask_location = mask_location, smoothing = 1, num_iters = 120)

Now lets color the segmentation mask by our cell type prediction.

In [4]:
# Compute cell categorization prediction for each cell
interior1 = cytoplasm_predictions[0,1,:,:]
interior2 = cytoplasm_predictions[0,2,:,:]
seg = label(cytoplasm_masks[0,:,:])
num_of_cells = np.amax(seg) 
prediction = np.zeros(interior1.shape, dtype = np.float32)
prediction_color = np.zeros((interior1.shape[0],interior1.shape[1],3), dtype = np.float32)

bound = segmentation.find_boundaries(seg)
for cell_no in xrange(1,num_of_cells):
	class_1_pred = interior1[seg == cell_no]
	class_2_pred = interior2[seg == cell_no]
	class_1_score = np.sum(class_1_pred)/(np.sum(class_1_pred) + np.sum(class_2_pred))
	class_2_score = np.sum(class_2_pred)/(np.sum(class_1_pred) + np.sum(class_2_pred))

	prediction[seg == cell_no] = class_2_score
	prediction_color[seg == cell_no,0] = plt.cm.coolwarm(class_2_score)[0]
	prediction_color[seg == cell_no,1] = plt.cm.coolwarm(class_2_score)[1]
	prediction_color[seg == cell_no,2] = plt.cm.coolwarm(class_2_score)[2]

prediction_color[bound,0] = 0
prediction_color[bound,1] = 0
prediction_color[bound,2] = 0
cnnout_name = os.path.join(mask_location, 'segmentation_rgb_new.tif')
scipy.misc.imsave(cnnout_name,np.float16(prediction_color))

Now lets quantify the classification accuracy

In [9]:
other_location = os.path.join(direc_name, 'OtherChannels')

imglist_class1 = nikon_getfiles(other_location,'CFP')
imglist_class2 = nikon_getfiles(other_location,'Far-red')

class1_name = os.path.join(other_location, imglist_class1[0])
class2_name = os.path.join(other_location, imglist_class2[0])

nuclear_class1 = get_image(class1_name) 
nuclear_class2 = get_image(class2_name)

nuclear_class1 -= np.mean(nuclear_class1)
nuclear_class2 -= np.mean(nuclear_class2)

truth = np.zeros(interior1.shape, dtype = np.float32)

correct_class1 = 0
incorrect_class1 = 0

correct_class2 = 0
incorrect_class2 = 0

for cell_no in xrange(1,num_of_cells):
	actual_class_1_pred = nuclear_class1[seg == cell_no]
	actual_class_2_pred = nuclear_class2[seg == cell_no]

	if np.sum(actual_class_1_pred) > np.sum(actual_class_2_pred):
		truth[seg == cell_no] = 1
	elif np.sum(actual_class_1_pred) < np.sum(actual_class_2_pred):
		truth[seg == cell_no] = 2

print num_of_cells
correct_scores = []
incorrect_scores = []

for cell_no in xrange(1,num_of_cells):
	prediction_value = np.float32(np.mean(prediction[seg == cell_no]) > 0.5) + 1
	truth_value = np.mean(truth[seg == cell_no])

	if truth_value == 1:
		if truth_value == prediction_value:
			correct_class1 += 1
			correct_scores += [1-np.mean(prediction[seg == cell_no])]
		elif truth_value != prediction_value:
			incorrect_class1 += 1
			incorrect_scores += [1-np.mean(prediction[seg == cell_no])]


	if truth_value == 2:
		if truth_value == prediction_value:
			correct_class2 += 1
			correct_scores += [np.mean(prediction[seg == cell_no])]

		elif truth_value != prediction_value:
			incorrect_class2 += 1
			incorrect_scores += [np.mean(prediction[seg == cell_no])]


print correct_class1, incorrect_class1
print correct_class2, incorrect_class2

correct_class1 = np.float32(correct_class1)
correct_class2 = np.float32(correct_class2)

percent_correct1 = correct_class1/(correct_class1 + incorrect_class1)*100
percent_correct2 = correct_class2/(correct_class2 +incorrect_class2)*100
percent_correct = [percent_correct1, percent_correct2]
print percent_correct

incorrect_hist = plt.hist(incorrect_scores, bins = 20, label = 'Incorrectly classified cells', color = (np.float(204)/255, np.float(121)/255, np.float(167)/255))
correct_hist = plt.hist(correct_scores, bins = 20, label = 'Correctly classified cells', color = (np.float(0)/255, np.float(114)/255, np.float(178)/255))

plt.xlabel(r'Cellular classification score', fontsize = 16)
plt.ylabel(r'Number of cells', fontsize = 16)
# plt.title('', y = 1.03, fontsize = 20)

plt.xlim([0, 1])
plt.xticks([0,0.5,1],  fontsize = 16)
plt.ylim([0, 150])
plt.yticks([0,50,100,150],  fontsize = 16)
plt.legend(loc = 2)
plt.tight_layout()
plt.savefig("class_scores.pdf")
plt.show()

294
84 13
196 0
[86.597938144329902, 100.0]
